SIAM - Discrete Bath
=======

We calculate the correlation functions 

$$\chi_{2,\sigma\sigma'}^{\rm PP} = \langle \left(c^\dagger_\sigma c^\dagger_{\sigma'}\right)(i\Omega) \left(c_\sigma c_{\sigma'}\right)(\tau=0) \rangle$$

and 

$$\chi_{2,\sigma\sigma'}^{\rm PH} = \langle n_\sigma (i\Omega) n_{\sigma'} (\tau=0) \rangle$$

for a SIAM with a discrete bath

$$h = h_0 + h_{\rm int} + h_{\rm bath} + h_{\rm coup}$$

where

$$h_0 = -\mu \, (n_{\uparrow, 0} + n_{\uparrow, 0}) - h(n_{\uparrow,0} - n_{\downarrow,0})$$

$$h_{\rm int} = U n_{\uparrow, 0} n_{\downarrow, 0}$$

$$h_{\rm bath} = \sum_{i=1}^{N} \sum_{\sigma} E_i n_{\sigma,i}$$

and

$$h_{\rm coup} = \sum_{i=1}^{N} \sum_{\sigma} V_i (c^\dagger_{\sigma,i} c_{\sigma,0} + c^\dagger_{\sigma,0} c_{\sigma,i})$$

The parameters are defined below.

In [ ]:
# %load model.py
import sys, os
sys.path.append(os.getcwd() + '/../common')
from util import *

from pytriqs.gf import Gf, MeshImFreq, iOmega_n, inverse
from pytriqs.operators import c, c_dag, n
from itertools import product

# ==== System Parameters ====
beta = 5.           # Inverse temperature
mu = 2.             # Chemical potential
U = 5.              # On-site density-density interaction
h = 0.2             # Local magnetic field
E = [ 0.0, 4.0 ]    # Bath-site energies
V = [ 2.0, 5.0 ]    # Couplings to Bath-sites

spin_names = ['up', 'dn']
orb_names  = [0]

# ==== Local Hamiltonian ====
h_0 = - mu*( n('up',0) + n('dn',0) ) - h*( n('up',0) - n('dn',0) )
h_int = U * n('up',0) * n('dn',0)
h_loc = h_0 + h_int

# ==== Bath & Coupling Hamiltonian ====
h_bath, h_coup = 0, 0
for i, E_i, V_i in zip([0, 1], E, V):
    for sig in ['up','dn']:
        h_bath += E_i * n(sig,'b_' + str(i))
        h_coup += V_i * (c_dag(sig,0) * c(sig,'b_' + str(i)) + c_dag(sig,'b_' + str(i)) * c(sig,0))

# ==== Total impurity hamiltonian and fundamental operators ====
h_imp = h_loc + h_coup + h_bath

# ==== Green function structure ====
gf_struct = [ [s, orb_names] for s in spin_names ]

# ==== Hybridization Function ====
n_iw = 20
iw_mesh = MeshImFreq(beta, 'Fermion', n_iw)
Delta = BlockGf(mesh=iw_mesh, gf_struct=gf_struct)
Delta << sum([V_i*V_i * inverse(iOmega_n - E_i) for V_i,E_i in zip(V, E)]);

# ==== Non-Interacting Impurity Green function  ====
G0_iw = BlockGf(mesh=iw_mesh, gf_struct=gf_struct)
G0_iw['up'] << inverse(iOmega_n + mu + h - Delta['up'])
G0_iw['dn'] << inverse(iOmega_n + mu - h - Delta['dn'])


Results
====

In [ ]:
from pytriqs.plot.mpl_interface import oplot, plt
%matplotlib inline

In [ ]:
# %load ../common/plot.py
import sys, os
sys.path.append(os.getcwd() + "/..")
sys.path.append(os.getcwd() + "/../../common")
from model import *

from pytriqs.archive import HDFArchive
from pytriqs.plot.mpl_interface import oplot, plt
from glob import glob
from os.path import basename

# === Load chi2pp and chi2ph for every solver

solver_lst = [ basename(f).strip('.h5') for f in glob('results/*.h5') ]
chi2pp_tau, chi2ph_tau = {}, {}

for solver in solver_lst:
    dat = HDFArchive('results/' + solver + '.h5','r')
    chi2pp_tau[solver] = dat['chi2pp_tau']
    chi2ph_tau[solver] = dat['chi2ph_tau']
    
# === For every block and solver, plot chi2pp and chi2ph

block_lst = [bl for bl in chi2pp_tau[solver_lst[0]].indices]
n_blocks = len(block_lst)

for chi2, name in [[chi2pp_tau, r'$\chi_2^{\rm PP}$'], [chi2ph_tau, r'$\chi_2^{\rm PH}$']]:

    for block in block_lst:

        if max(norm_inf(chi2[solver][block][0,0,0,0]) for solver in solver_lst) < 1e-14: continue
        
        plt.figure(figsize=(10,6))
        for solver in solver_lst:
            oplot(chi2[solver][block][0,0,0,0], name = name + "_%s" % solver)

        plt.suptitle(name + "[" + str(block) + r"][0,0,0,0]", fontsize=14, y=1.05)
        plt.xlabel(r"$\tau$")

        plt.tight_layout()
        plt.show()


Deviations
======

We present a table containing deviations between the different solvers measured via

$$||G_{\rm Solver_1} - G_{\rm Solver_2}||_\infty$$

In [ ]:
for chi2, name in [[chi2pp_tau, 'chi_2^{PP}'], [chi2ph_tau, 'chi_2^{PH}']]:

    for block in block_lst:
        
        if max(norm_inf(chi2[solver][block]) for solver in solver_lst) < 1e-14: continue
        
        deviations = [[ norm_inf(chi2[s1][block].data - chi2[s2][block].data) for s1 in solver_lst ] \
                        for s2 in solver_lst ]
    
        print "\t\t    Deviations for " + name + str(block)
        print "\t\t -----------------------------------"

        row_format ="{:>15}" * (len(solver_lst) + 1)
        print row_format.format("", *solver_lst)
        row_format ="{:>15}" + "{:>15.2E}" * len(solver_lst)
        for solver, row in zip(solver_lst, deviations):
            print row_format.format(solver, *row)

        print "\n\n"